In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
import sys
sys.path.append("../Python")
import ROOT as R
import time
import numpy as np
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
from root_helpers import fancy_plot
from root_helpers import print_mc_particle_tree
from root_helpers import print_daughters
R.EnableImplicitMT()
import os
recompile = True
try:
    if os.path.getmtime('../Python/Utility_Functions_C.so') - os.path.getmtime('../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../Python/Utility_Functions_C.so")
R.Utility_Functions()

Welcome to JupyROOT 6.29/01
Recompile is not needed


'Utility Functions V1.0.5 \n'

In [2]:
fiducial_cut = False
data_type = "_" # "_uncor_"
data_type_name = "Corrected"
if fiducial_cut:
    fiducial_cut_name = "fiducial"
else:
    fiducial_cut_name = ""

ch = R.TChain("MiniDST")
data_source="/data/HPS/data/physrun2021/MC_RAW"
energy="*"
particle="e-"
if particle == "e+":
    particle_s = "e\+"
else:
    particle_s = particle
if energy == "*":
    energy_s = "all_"
else:
    energy_s = energy;
data_source_glob = data_source+"/hpsForward_"+particle_s+"_"+energy+"GeV_*_SLIC-*_recon.root"
print(f"Adding all data from: {data_source_glob}")
# ch.Add(data_source_glob)
ch.Add("/data/HPS/data/physrun2019/MC_RAW/hpsForward_e-_3.0GeV_z-7.5_x_SLIC-v06-00-01_QGSP_BERT_HPS-PhysicsRun2019-v2-4pt5_recon.root")           # sim_2019/hpsForward_e-_3.0GeV_*.root")

print(f"Number of events for {particle} = {ch.GetEntries()/1e6:7.2f} M")
df = R.RDataFrame(ch)

dfx= df.Define("n_kf_track","int n=0; for(int i=0;i<track_type.size();++i){if(track_type[i]==1) n+=1;}; return n") \
    .Filter("n_kf_track==1").Filter("ecal_cluster"+data_type+"energy.size()==1")

# if fiducial_cut:
#     dfx = dfx.Define("is_fiducial","is_in_fiducial_region(ecal_cluster_seed_ix[0], ecal_cluster_seed_iy[0])") \
#         .Filter("is_fiducial == true") \

dfx= dfx \
     .Define("trk_idx","for(int i=0;i<track_type.size();++i){if(track_type[i]==1) return(i);} return(-1);") \
     .Define("trk_gbl_idx","for(int i=0;i<track_type.size();++i){if(track_type[i]>=32) return(i);} return(-1);") \
     .Define("diff_posx_ecal","ecal_cluster"+data_type+"x[0] - track_x_at_ecal[trk_idx]") \
     .Define("diff_posy_ecal","ecal_cluster"+data_type+"y[0] - track_y_at_ecal[trk_idx]") \
     .Define("diff_E_p","ecal_cluster"+data_type+"energy[0] - get_vec_abs(track_px,track_py,track_pz)[trk_idx]") \
     .Define("diff_p_kf_gbl","get_vec_abs(track_px,track_py,track_pz)[trk_idx] - get_vec_abs(track_px,track_py,track_pz)[trk_gbl_idx]") \
     .Define("mc_part_primary_z","find_end_z_of_primary_mc_part(mc_part_z, mc_part_end_z)") \
     .Define("mc_part_daughters_z","find_average_end_of_daughters_of_primary_mc_part(mc_part_z, mc_part_end_z, mc_part_daughters)")

dfx = dfx.Define("score_energy","score_plane_hit_energy(mc_score_type, mc_score_px, mc_score_py, mc_score_pz, mc_score_x, mc_score_y, mc_score_z)") \
      .Define("score_x","score_plane_hit_x(mc_score_type, mc_score_px, mc_score_py, mc_score_pz, mc_score_x, mc_score_y, mc_score_z)") \
      .Define("score_y","score_plane_hit_y(mc_score_type, mc_score_px, mc_score_py, mc_score_pz, mc_score_x, mc_score_y, mc_score_z)") \
      .Define("n_score","score_energy.size()") \
      .Define("n_cluster","ecal_cluster_energy.size()")

dfxx= dfx.Filter("(n_score == 1) && (n_cluster == 1)") \
     .Define("score_ecal_diff_x","score_x[0] - ecal_cluster"+data_type+"x[0]") \
     .Define("score_ecal_diff_y","score_y[0] - ecal_cluster"+data_type+"y[0]") \
     .Define("score_ecal_diff_energy","score_energy[0] - ecal_cluster"+data_type+"energy[0]") \
     .Define("score_track_diff_x","score_x[0] - track_x_at_ecal[trk_idx]") \
     .Define("score_track_diff_y","score_y[0] - track_y_at_ecal[trk_idx]")

hx_diff_posx_ecal=dfx.Histo1D(("hx_diff_posx",data_type_name+" Diff Ecal_x - Track_x;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_top=dfx.Filter("ecal_cluster"+data_type+"y[0]>0").Histo1D(("hx_diff_posx_top",data_type_name+" Diff Ecal_x - Track_x Top;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_bot=dfx.Filter("ecal_cluster"+data_type+"y[0]<0").Histo1D(("hx_diff_posx_bot",data_type_name+" Diff Ecal_x - Track_x Bot;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_L=dfx.Filter("ecal_cluster"+data_type+"x[0]<0").Histo1D(("hx_diff_posx_L",data_type_name+" Diff Ecal_x - Track_x L;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_R=dfx.Filter("ecal_cluster"+data_type+"x[0]>0").Histo1D(("hx_diff_posx_R",data_type_name+" Diff Ecal_x - Track_x R;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posy_ecal=dfx.Histo1D(("hx_diff_posy",data_type_name+" Diff Ecal_y - Track_y;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_top=dfx.Filter("ecal_cluster"+data_type+"y[0]>0").Histo1D(("hx_diff_posy_top",data_type_name+" Diff Ecal_y - Track_y Top;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_bot=dfx.Filter("ecal_cluster"+data_type+"y[0]<0").Histo1D(("hx_diff_posy_bot",data_type_name+" Diff Ecal_y - Track_y Bot;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_L=dfx.Filter("ecal_cluster"+data_type+"x[0]<0").Histo1D(("hx_diff_posy_L",data_type_name+" Diff Ecal_y - Track_y L;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_R=dfx.Filter("ecal_cluster"+data_type+"x[0]>0").Histo1D(("hx_diff_posy_R",data_type_name+" Diff Ecal_y - Track_y R;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_e_p = dfx.Histo1D(("hx_diff_e_p", data_type_name+" Diff Energy - Momentum;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_top = dfx.Filter("ecal_cluster"+data_type+"y[0]>0").Histo1D(("hx_diff_e_p_top", data_type_name+" Diff Energy - Momentum Top;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_bot = dfx.Filter("ecal_cluster"+data_type+"y[0]<0").Histo1D(("hx_diff_e_p_bot", data_type_name+" Diff Energy - Momentum Bot;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_L = dfx.Filter("ecal_cluster"+data_type+"x[0]<0").Histo1D(("hx_diff_e_p_L", data_type_name+" Diff Energy - Momentum Left;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_R = dfx.Filter("ecal_cluster"+data_type+"x[0]>0").Histo1D(("hx_diff_e_p_R",data_type_name+" Diff Energy - Momentum Right;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
# hx_diff_p_kf_gbl = dfx.Histo1D(("hx_diff_p_kf_gbl",data_type_name+" Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
# hx_diff_p_kf_gbl_top = dfx.Filter("ecal_cluster"+data_type+"y[0]>0").Histo1D(("hx_diff_p_kf_gbl_top",data_type_name+" Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
# hx_diff_p_kf_gbl_bot = dfx.Filter("ecal_cluster"+data_type+"y[0]<0").Histo1D(("hx_diff_p_kf_gbl_bot",data_type_name+" Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
# hx_diff_p_kf_gbl_L = dfx.Filter("ecal_cluster"+data_type+"x[0]<0").Histo1D(("hx_diff_p_kf_gbl_L",data_type_name+" Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
# hx_diff_p_kf_gbl_R = dfx.Filter("ecal_cluster"+data_type+"x[0]>0").Histo1D(("hx_diff_p_kf_gbl_R",data_type_name+" Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_mc_part_primary_z = dfx.Histo1D(("hx_mc_part_primary_z","End of track for primary MC particle.;z[mm]",251,1349.5,1600.5),"mc_part_primary_z")
hx_mc_part_ave_daughter_z = dfx.Histo1D(("hx_mc_part_ave_daughter_z","Average end of track for primary MC particle daughters.;z[mm]",251,1349.5,1600.5),"mc_part_daughters_z")

hx_track_z_at_ecal = dfx.Histo1D(("hx_track_z_at_ecal","Track z at ECal; z[mm]", 301,1299.5,1500.5),"track_z_at_ecal")
hx_ecal_cluster_z = dfx.Histo1D(("hx_ecal_cluster_z","Ecal cluster z position;z [mm]",301,1299.5,1500.5),"ecal_cluster_z")
hx_ecal_cluster_uncor_z = dfx.Histo1D(("hx_ecal_cluster_uncor_z","Ecal cluster z position;z [mm]",301,1299.5,1500.5),"ecal_cluster_uncor_z")

# Scoring Plane
# hxx_diff_score_ecal_posx = dfx.Histo1D(("hxx_diff_score_ecal_posx","Score plane x - "+data_type_name+" Ecal_x;#Delta x [mm]", 1000,-20.,20.),"score_ecal_diff_x")
# hxx_diff_score_ecal_posx_top = dfx.Filter("ecal_cluster"+data_type+"y[0]>0").Histo1D(("hxx_diff_score_ecal_posx_top","Score plane x - "+data_type_name+" Ecal_x;#Delta x [mm]", 1000,-20.,20.),"score_ecal_diff_x")
# hxx_diff_score_ecal_posx_bot = dfx.Filter("ecal_cluster"+data_type+"y[0]<0").Histo1D(("hxx_diff_score_ecal_posx_bot","Score plane x - "+data_type_name+" Ecal_x;#Delta x [mm]", 1000,-20.,20.),"score_ecal_diff_x")
# hxx_diff_score_ecal_posx_L = dfx.Filter("ecal_cluster"+data_type+"x[0]<0").Histo1D(("hxx_diff_score_ecal_posx_L","Score plane x - "+data_type_name+" Ecal_x;#Delta x [mm]", 1000,-20.,20.),"score_ecal_diff_x")
# hxx_diff_score_ecal_posx_R = dfx.Filter("ecal_cluster"+data_type+"x[0]>0").Histo1D(("hxx_diff_score_ecal_posx_R","Score plane x - "+data_type_name+" Ecal_x;#Delta x [mm]", 1000,-20.,20.),"score_ecal_diff_x")
#
# hxx_diff_score_ecal_posy = dfx.Histo1D(("hxx_diff_score_ecal_posy","Score plane y - "+data_type_name+" Ecal_y;#Delta y [mm]", 1000,-20.,20.),"score_ecal_diff_y")
# hxx_diff_score_ecal_posy_top = dfx.Filter("ecal_cluster"+data_type+"y[0]>0").Histo1D(("hxx_diff_score_ecal_posy_top","Score plane y - "+data_type_name+" Ecal_y;#Delta y [mm]", 1000,-20.,20.),"score_ecal_diff_y")
# hxx_diff_score_ecal_posy_bot = dfx.Filter("ecal_cluster"+data_type+"y[0]<0").Histo1D(("hxx_diff_score_ecal_posy_bot","Score plane y - "+data_type_name+" Ecal_y;#Delta y [mm]", 1000,-20.,20.),"score_ecal_diff_y")
# hxx_diff_score_ecal_posy_L = dfx.Filter("ecal_cluster"+data_type+"x[0]<0").Histo1D(("hxx_diff_score_ecal_posy_L","Score plane y - "+data_type_name+" Ecal_y;#Delta y [mm]", 1000,-20.,20.),"score_ecal_diff_y")
# hxx_diff_score_ecal_posy_R = dfx.Filter("ecal_cluster"+data_type+"x[0]>0").Histo1D(("hxx_diff_score_ecal_posy_R","Score plane y - "+data_type_name+" Ecal_y;#Delta y [mm]", 1000,-20.,20.),"score_ecal_diff_y")
#
#
#
# hxx_diff_score_track_posx = dfx.Histo1D(("hxx_diff_score_track_posx","Score plane x - Track x;#Delta x [mm]", 1000,-20.,20.),"score_track_diff_x")
# hxx_diff_score_track_posx_top = dfx.Filter("track_y_at_ecal[trk_idx]>0").Histo1D(("hxx_diff_score_track_posx","Score plane x - Track x;#Delta x [mm]", 1000,-20.,20.),"score_track_diff_x")
# hxx_diff_score_track_posx_bot = dfx.Filter("track_y_at_ecal[trk_idx]<0").Histo1D(("hxx_diff_score_track_posx","Score plane x - Track x;#Delta x [mm]", 1000,-20.,20.),"score_track_diff_x")
# hxx_diff_score_track_posx_L = dfx.Filter("track_x_at_ecal[trk_idx]<0").Histo1D(("hxx_diff_score_track_posx","Score plane x - Track x;#Delta x [mm]", 1000,-20.,20.),"score_track_diff_x")
# hxx_diff_score_track_posx_R = dfx.Filter("track_x_at_ecal[trk_idx]>0").Histo1D(("hxx_diff_score_track_posx","Score plane x - Track x;#Delta x [mm]", 1000,-20.,20.),"score_track_diff_x")
#
# hxx_diff_score_track_posy = dfx.Histo1D(("hxx_diff_score_track_posy","Score plane y - Track y;#Delta y [mm]", 1000,-20.,20.),"score_track_diff_y")
# hxx_diff_score_track_posy_top = dfx.Filter("track_y_at_ecal[trk_idx]>0").Histo1D(("hxx_diff_score_track_posy","Score plane y - Track y;#Delta y [mm]", 1000,-20.,20.),"score_track_diff_y")
# hxx_diff_score_track_posy_bot = dfx.Filter("track_y_at_ecal[trk_idx]<0").Histo1D(("hxx_diff_score_track_posy","Score plane y - Track y;#Delta y [mm]", 1000,-20.,20.),"score_track_diff_y")
# hxx_diff_score_track_posy_L = dfx.Filter("track_x_at_ecal[trk_idx]<0").Histo1D(("hxx_diff_score_track_posy","Score plane y - Track y;#Delta y [mm]", 1000,-20.,20.),"score_track_diff_y")
# hxx_diff_score_track_posy_R = dfx.Filter("track_x_at_ecal[trk_idx]>0").Histo1D(("hxx_diff_score_track_posy","Score plane y - Track y;#Delta y [mm]", 1000,-20.,20.),"score_track_diff_y")

Adding all data from: /data/HPS/data/physrun2021/MC_RAW/hpsForward_e-_*GeV_*_SLIC-*_recon.root
Number of events for e- =    0.63 M


In [11]:
%%time
cc5 = R.TCanvas("cc5","Canvas",800,600)
hx_mc_part_primary_z.SetStats(0)
hx_mc_part_ave_daughter_z.SetStats(0)
hx_mc_part_primary_z.SetTitle("Z positions of tracks and ECal clusters")
hx_mc_part_primary_z.Draw()

hx_mc_part_ave_daughter_z.SetLineColor(R.kRed)
hx_mc_part_ave_daughter_z.SetLineWidth(2)
hx_mc_part_ave_daughter_z.Draw("same")
l1 = R.TLine(1448, 0, 1448, 1.05*hx_mc_part_primary_z.GetMaximum())
l1.SetLineWidth(2)
l1.SetLineColor(R.kOrange)

l1.Draw()
hx_mc_part_primary_z.SetLineColor(R.kBlue)
hx_mc_part_primary_z.Draw("same")

hx_ecal_cluster_z.SetLineColor(R.kGreen+1)
hx_ecal_cluster_z.SetFillColor(R.kGreen+1)
hx_ecal_cluster_z.Draw("hist same")

leg5 = R.TLegend(0.1,0.7,0.4,0.9)
leg5.AddEntry(hx_mc_part_primary_z.GetValue(),"Endpoint of primary MC particle")
leg5.AddEntry(hx_mc_part_ave_daughter_z.GetValue(),"Average endpoint of daughters.")
leg5.AddEntry(hx_ecal_cluster_z.GetValue(),"Ecal Cluster z")
leg5.AddEntry(l1,"Ecal face.")
leg5.Draw()
cc5.Draw()
cc5.SaveAs("end_of_track_and_ecal_z.pdf")
print(f"Mean of endpoint primary MC particles: {hx_mc_part_primary_z.GetMean()}")
print(f"Mean of endpoint daughter particles: {hx_mc_part_ave_daughter_z.GetMean()}")
print(f"Mean of endpoint ecal clusters: {hx_ecal_cluster_z.GetMean()}")

Mean of endpoint primary MC particles: 1482.5962312563745
Mean of endpoint daughter particles: 1469.3926631056165
Mean of endpoint ecal clusters: 1449.2084823522423
CPU times: user 51.8 ms, sys: 3.67 ms, total: 55.5 ms
Wall time: 55 ms


Info in <TCanvas::Print>: pdf file end_of_track_and_ecal_z.pdf has been created


In [12]:
%%time
cc5a = R.TCanvas("cc5a","Canvas",800,600)
hx_track_z_at_ecal.SetStats(0)
hx_track_z_at_ecal.SetLineColor(R.kBlue)
hx_track_z_at_ecal.SetFillColor(R.kBlue)
xax = hx_track_z_at_ecal.GetXaxis()
xax.SetLimits(1440,1460)
hx_track_z_at_ecal.Draw()
hx_ecal_cluster_z.SetLineColor(R.kGreen)
hx_ecal_cluster_z.SetLineWidth(2)
hx_ecal_cluster_z.Draw("same")
track_z_at_ecal_mean = hx_track_z_at_ecal.GetMean()
cc5a.Draw()
print(f"Track z at ecal = {track_z_at_ecal_mean} mm")
print(f"N hx_track_z_at_ecal = {hx_track_z_at_ecal.GetEntries()/1e6} M, N hx_ecal_cluster = {hx_ecal_cluster_z.GetEntries()/1e6} M")

Track z at ecal = 1448.0 mm
N hx_track_z_at_ecal = 1.122149 M, N hx_ecal_cluster = 0.381228 M
CPU times: user 13.2 ms, sys: 1.13 ms, total: 14.3 ms
Wall time: 13.9 ms


In [9]:
%%time
hx_nscore = dfx.Histo1D(("hx_nscore","N scoring plane cluster",11,-0.5,10.5),"n_score")
hx_n_mc_score = df.Define("n_mc_score","mc_score_type.size()").Histo1D(("hx_n_mc_score","N scoring plane",351,-0.5,350.5),"n_mc_score")
hx_ncluster = df.Define("n_clusters","ecal_cluster_energy.size()").Histo1D(("hx_ncluster","N clusters", 11,-0.5,10.5),"n_clusters")
cc1 = R.TCanvas("cc1","cc1", 1200,600)
cc1.Divide(2,1)
p1 = cc1.cd(1)
p1.SetLogy()
hx_nscore.SetLineColor(R.kGreen)
hx_nscore.Draw()
hx_ncluster.SetLineColor(R.kRed)
hx_ncluster.Draw("same")
p2 = cc1.cd(2)
p2.SetLogy()
hx_n_mc_score.SetLineColor(R.kBlue)
hx_n_mc_score.Draw()
cc1.Draw()

CPU times: user 7.07 s, sys: 200 ms, total: 7.27 s
Wall time: 1.31 s


Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc1


In [13]:
cc4 = R.TCanvas("cc4","Canvas",1200,600)
cc4.Divide(2,2)
p1=cc4.cd(1)
#p1.SetLogy()
hx_diff_posx_ecal.SetStats(0)
hx_diff_posx_ecal.Draw()
hx_diff_posx_ecal_top.SetLineColor(R.kGreen)
hx_diff_posx_ecal_top.Draw("same")
hx_diff_posx_ecal_bot.SetLineColor(R.kRed)
hx_diff_posx_ecal_bot.Draw("same")
hx_diff_posx_ecal_L.SetLineColor(R.kMagenta)
hx_diff_posx_ecal_L.Draw("same")
hx_diff_posx_ecal_R.SetLineColor(R.kCyan)
hx_diff_posx_ecal_R.Draw("same")
p2=cc4.cd(2)
#p2.SetLogy()
hx_diff_posy_ecal.SetStats(0)
hx_diff_posy_ecal.Draw()
hx_diff_posy_ecal_top.SetLineColor(R.kGreen)
hx_diff_posy_ecal_top.Draw("same")
hx_diff_posy_ecal_bot.SetLineColor(R.kRed)
hx_diff_posy_ecal_bot.Draw("same")
hx_diff_posy_ecal_L.SetLineColor(R.kMagenta)
hx_diff_posy_ecal_L.Draw("same")
hx_diff_posy_ecal_R.SetLineColor(R.kCyan)
hx_diff_posy_ecal_R.Draw("same")
p3=cc4.cd(3)
# p3.SetLogy()
hx_diff_e_p.SetStats(0)
hx_diff_e_p.Draw()
hx_diff_e_p_top.SetLineColor(R.kGreen)
hx_diff_e_p_top.Draw("same")
hx_diff_e_p_bot.SetLineColor(R.kRed)
hx_diff_e_p_bot.Draw("same")
hx_diff_e_p_L.SetLineColor(R.kMagenta)
hx_diff_e_p_L.Draw("same")
hx_diff_e_p_R.SetLineColor(R.kCyan)
hx_diff_e_p_R.Draw("same")
p4=cc4.cd(4)
# p4.SetLogy()
# hx_diff_p_kf_gbl.SetStats(0)
# hx_diff_p_kf_gbl.Draw()
# hx_diff_p_kf_gbl_top.SetLineColor(R.kGreen)
# hx_diff_p_kf_gbl_top.Draw("same")
# hx_diff_p_kf_gbl_bot.SetLineColor(R.kRed)
# hx_diff_p_kf_gbl_bot.Draw("same")
# hx_diff_p_kf_gbl_L.SetLineColor(R.kMagenta)
# hx_diff_p_kf_gbl_L.Draw("same")
# hx_diff_p_kf_gbl_R.SetLineColor(R.kCyan)
# hx_diff_p_kf_gbl_R.Draw("same")

# h_trk_type.Draw()
cc4.cd(0)
leg1 = R.TLegend(0.8,0.8,0.94,0.94)
leg1.AddEntry(hx_diff_posx_ecal.GetValue(), "All data with one cluster")
leg1.AddEntry(hx_diff_posx_ecal_top.GetValue(),"Top only")
leg1.AddEntry(hx_diff_posx_ecal_bot.GetValue(),"Bottom only")
leg1.AddEntry(hx_diff_posx_ecal_L.GetValue(),"Left Only x<0")
leg1.AddEntry(hx_diff_posx_ecal_R.GetValue(),"Right Only x>0")
leg1.Draw()

cc4_text = R.TPaveText(0.43,0.93,0.57,0.99)
cc4_text.AddText(f"Track endpoint = {track_z_at_ecal_mean:6.1f}")
cc4_text.Draw()

cc4.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc4
